In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12-0.13.5") \
    .getOrCreate()

In [0]:
df = spark.read.format("com.crealytics.spark.excel") \
            .option("inferSchema", "false") \
            .option("header", "true") \
            .option("dataAddress", "'Dirty 8'!A1") \
            .load("/FileStore/tables/8__Invoices_with_Merged_Categories_and_Merged_Amounts.xlsx") 

df.show(truncate=False)

+--------------+------------------------------------------+---------------------------------------+
|Order ID      |Category                                  |Amount                                 |
+--------------+------------------------------------------+---------------------------------------+
|CA-2011-167199|Binders | Art | Phones | Fasteners | Paper|609.98 | 5.48 | 391.98 | 755.96 | 31.12|
|CA-2011-149020|Office Supplies | Furniture               |2.98 | 51.94                           |
|CA-2011-131905|Office Supplies | Technology | Technology |7.2 | 42.0186 | 42.035                 |
|CA-2011-127614|Accessories | Tables | Binders            |234.45 | 1256.22 | 17.46               |
|CA-2011-112352|Stationery | Furniture                    |54.35                                  |
|CA-2011-105898|Art                                       |5.48 | 65.58                           |
+--------------+------------------------------------------+---------------------------------------+


In [0]:
from pyspark.sql.functions import split, posexplode, coalesce, col

In [0]:
# '|' : it will split every character
# '[|]': it split only the character between '|'
df = df.withColumn("splitted_category", split('category', '[|]')).drop("category")

In [0]:
df = df.withColumn("splitted_amount", split('amount', '[|]')).drop("amount")

In [0]:
df.show(truncate=False)

+--------------+------------------------------------------------+---------------------------------------------+
|Order ID      |splitted_category                               |splitted_amount                              |
+--------------+------------------------------------------------+---------------------------------------------+
|CA-2011-167199|[Binders ,  Art ,  Phones ,  Fasteners ,  Paper]|[609.98 ,  5.48 ,  391.98 ,  755.96 ,  31.12]|
|CA-2011-149020|[Office Supplies ,  Furniture]                  |[2.98 ,  51.94]                              |
|CA-2011-131905|[Office Supplies ,  Technology ,  Technology]   |[7.2 ,  42.0186 ,  42.035]                   |
|CA-2011-127614|[Accessories ,  Tables ,  Binders]              |[234.45 ,  1256.22 ,  17.46]                 |
|CA-2011-112352|[Stationery ,  Furniture]                       |[54.35]                                      |
|CA-2011-105898|[Art]                                           |[5.48 ,  65.58]                        

In [0]:
exploded_cat = df.selectExpr("*", "posexplode(splitted_category) as (pos,col)")
exploded_amt = df.selectExpr("*", "posexplode(splitted_amount) as (pos,col1)")

df_combined = exploded_cat.join(exploded_amt, on=["Order ID", "pos"], how="full_outer")

In [0]:
df_combined.show(truncate=False)

+--------------+---+------------------------------------------------+---------------------------------------------+----------------+------------------------------------------------+---------------------------------------------+---------+
|Order ID      |pos|splitted_category                               |splitted_amount                              |col             |splitted_category                               |splitted_amount                              |col1     |
+--------------+---+------------------------------------------------+---------------------------------------------+----------------+------------------------------------------------+---------------------------------------------+---------+
|CA-2011-105898|0  |[Art]                                           |[5.48 ,  65.58]                              |Art             |[Art]                                           |[5.48 ,  65.58]                              |5.48     |
|CA-2011-105898|1  |null                        

In [0]:
df_final = df_combined.select(
    "Order ID", col("col").alias("Category"), col("col1").alias("Amount")
)

+--------------+----------------+---------+
|      Order ID|        Category|   Amount|
+--------------+----------------+---------+
|CA-2011-105898|             Art|    5.48 |
|CA-2011-105898|            null|    65.58|
|CA-2011-112352|     Stationery |    54.35|
|CA-2011-112352|       Furniture|     null|
|CA-2011-127614|    Accessories |  234.45 |
|CA-2011-127614|         Tables | 1256.22 |
|CA-2011-127614|         Binders|    17.46|
|CA-2011-131905|Office Supplies |     7.2 |
|CA-2011-131905|     Technology | 42.0186 |
|CA-2011-131905|      Technology|   42.035|
|CA-2011-149020|Office Supplies |    2.98 |
|CA-2011-149020|       Furniture|    51.94|
|CA-2011-167199|        Binders |  609.98 |
|CA-2011-167199|            Art |    5.48 |
|CA-2011-167199|         Phones |  391.98 |
|CA-2011-167199|      Fasteners |  755.96 |
|CA-2011-167199|           Paper|    31.12|
+--------------+----------------+---------+



In [0]:
display(df_final)

Order ID,Category,Amount
CA-2011-105898,Art,5.48
CA-2011-105898,null,65.58
CA-2011-112352,Stationery,54.35
CA-2011-112352,Furniture,null
CA-2011-127614,Accessories,234.45
CA-2011-127614,Tables,1256.22
CA-2011-127614,Binders,17.46
CA-2011-131905,Office Supplies,7.2
CA-2011-131905,Technology,42.0186
CA-2011-131905,Technology,42.035


In [0]:
df_final.columns

Out[14]: ['Order ID', 'Category', 'Amount']

In [0]:
df_final.filter(df_final["Amount"].isNull()).show()

+--------------+----------+------+
|      Order ID|  Category|Amount|
+--------------+----------+------+
|CA-2011-112352| Furniture|  null|
+--------------+----------+------+



In [0]:
df_final.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Amount: string (nullable = true)



In [0]:
df_final = df_final.withColumn("Amount", col("Amount").cast("decimal(10,2)"))

In [0]:
df_final.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Amount: decimal(10,2) (nullable = true)



In [0]:
df_final = df_final.na.fill(value=0, subset=["Amount"])